# 2 Задание

In [ ]:
from PIL import Image
from matplotlib.pyplot import figure, imshow, show
import numpy as np

def get_image() -> np.array:
    dir = "images/"
    image = Image.open(dir + "example.png")
    image_array = np.array(image)[:, :, :3]
    return image_array


def show_image(array: np.array):
    figure()
    imshow(array, cmap='Greys')
   

# show_image(get_image())

## Полутоновое изображение

In [ ]:
def get_halftone_image() -> np.array:
    image_array = get_image()
    mean = np.mean(image_array, axis=2, dtype=int)
    return mean

## Шум

In [ ]:
import random


def get_noise_image(mean: float = 50, sigma: float = 5, data_type=float) -> np.array:
    halftone_image_array = get_halftone_image()
    return np.clip(np.array([np.array([pixel + random.gauss(mean, sigma) * random.randint(0, 1) for pixel in row], dtype=data_type)
                     for row in halftone_image_array]), 0, 255)

## Гистограмма

In [ ]:
from collections import Counter

from matplotlib.pyplot import bar
import numpy.typing as npt


def get_hist(data: npt.NDArray[int]) -> npt.NDArray[int]:
    cnt = Counter(data.reshape(-1))
    y = np.array([cnt.get(el, 0) for el in range(256)])
    return y


def draw_hist(data: npt.NDArray[int]):
    x = list(range(256))
    y = get_hist(data)
    bar(x, y)

# noise_appended_array = get_noise_image(data_type=int)
# draw_hist(get_halftone_image())

## Размытие Гаусса

In [ ]:
def image_filter(K_size, sigma, im):
    H, W = im.shape
    
    pad = K_size // 2
    out = np.zeros((H + pad * 2, W + pad * 2), np.float64)
    out[pad: pad + H, pad: pad + W] = im.copy().astype(np.float64)

    K = np.zeros((K_size, K_size), np.float64)
    for x in range(-pad, -pad + K_size):
        for y in range(-pad, -pad + K_size):
            K[x + pad, y + pad] = np.exp( -(x * x + y * y) / (2 * (sigma * sigma)))
    K /= 2 * np.pi * sigma * sigma
    K /= K.sum()
 
    tmp = out.copy()
  
    for y in range(H):
        for x in range(W):
            out[pad + y, pad + x] = np.sum(K * tmp[y: y + K_size, x: x + K_size])
    out = np.clip(out, 0, 255).astype(int)
    return out[pad: pad + H, pad: pad + W]

## Бинаризация изображения

In [ ]:
def otsu_algorithm(hist: npt.NDArray[int], t: int):
    T = u = t
    s_max = 0
    freq = np.cumsum(hist)
    while u < 255:
        q1 = freq[u]
        q2 = freq[255] - freq[u]
        mub = np.mean(hist[:u])
        muf = np.mean(hist[u:])
        dw = q1 * q2 * (mub - muf) ** 2
        if dw > s_max:
            s_max = dw
            T = u
        u += t
    return T


def salt_and_pepper_filter(image):
    pass


def get_binary_image():
    # image = image_filter(10, 2.8, get_noise_image())
    # image = get_halftone_image()
    image = get_noise_image()
    t = otsu_algorithm(get_hist(image), 1)
    image[image <= t] = t
    image[image > t] = 0
    image[image == t] = 1
    return image

show_image(get_binary_image())